In [ ]:
%pip install huggingface_hub
%pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu128
%pip install transformers
%pip install datasets, evaluate

from huggingface_hub import snapshot_download, login
import os

token = ""
#Your HF token.
login(token=token)

local_dir = snapshot_download("google/gemma-3-12b-it")
print("Files are in:", local_dir)

^C
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.



KeyboardInterrupt


KeyboardInterrupt


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: 'datasets,': Expected end or semicolon (after name and no valid version specifier)
    datasets,
            ^


Looking in indexes: https://download.pytorch.org/whl/nightly/cu128



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/root/.cache/huggingface/hub/models--google--gemma-3-12b-it/snapshots/96b6f1eccf38110c56df3a15bffe176da04bfd80"
#Or whatever path the model was downloaded to.

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16).to("cuda")

In [ ]:
model = model.to("cuda").eval()
for param in model.parameters():
    param.requires_grad = False
#We're not actually changing anything in the model itself.

In [ ]:
del(model.model.vision_tower)
del(model.model.multi_modal_projector)

lm = model.model.language_model
layer_list = []
for i in range(24):
##for i in range(1):
    layer_list.append(lm.layers[i])
lm.layers = torch.nn.ModuleList(layer_list)

del(model.lm_head)
#Keeping things tidy, delete everything we don't need.

In [ ]:
import copy
def reset_weights(m):
    if hasattr(m, 'reset_parameters'):
        m.reset_parameters()

new_layer = copy.deepcopy(model.model.language_model.layers[23]).to("cuda")
##new_layer = copy.deepcopy(model.model.language_model.layers[0]).to("cuda")
#Not that it really matters.
new_layer.config.use_cache = False
#Turn off KV caching for training.
new_layer.apply(reset_weights)
#No cheating!
for param in new_layer.parameters():
    param.requires_grad = True
ckpt_path = "epoch_04_rd2.pt"
checkpoint = torch.load(ckpt_path, map_location="cuda")
with torch.no_grad():
    new_layer.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
decay, no_decay = [], []
for n, p in new_layer.named_parameters():
    if p.ndim > 1:
        decay.append(p)
    else:
        no_decay.append(p)

optim = torch.optim.AdamW([{"params": decay, "weight_decay": 0.1}, {"params": no_decay,  "weight_decay": 0.0}], lr=3e-4)
#Weight decay apparently works best on matrix parameters and not scalar parameters like those that appear in layernorm.
#I'm a little skeptical that this really matters, but sure, we'll play the game.

In [ ]:
pred_latent = None
gold_latent = None

def hook_fn(mod, args, kwargs, output):
    #We're going to attach this hook to the layer we're copying.
    #We take its input, feed it into the original layer to get the gold output, and feed it into our copied layer to get the predicted output.
    global pred_latent, gold_latent

    inp = args[0]
    peg = kwargs["position_embeddings_global"]
    pel = kwargs["position_embeddings_local"]
    #Gemma really likes its position embeddings.

    attn_m = kwargs.get("attention_mask", None)
    pos_ids = kwargs.get("position_ids",    None)

    pred_latent = new_layer(
        inp, peg, pel,
        attention_mask=attn_m,
        position_ids=pos_ids,
        use_cache=False,
        past_key_value=None,
        output_attentions=False,
    )[0]

    gold_latent = output[0].detach()

In [ ]:
def tokenize(batch):
    return tokenizer(batch["text"])

In [ ]:
from datasets import concatenate_datasets, load_dataset
from datasets import interleave_datasets

wiki   = load_dataset("wikimedia/wikipedia", "20231101.en",  split="train", streaming=True)
books  = load_dataset("lucadiliello/bookcorpusopen", split="train", streaming=True)
crawl  = load_dataset("allenai/c4", "en", split="train", streaming=True)
#A decent enough mix for a quick run, I think.

mix = interleave_datasets([wiki, books, crawl], probabilities=[0.3, 0.3, 0.4])
ds_tok = mix.map(tokenize, batched=True, remove_columns=["text"])

In [ ]:
def group_texts(batch, chunk=2048):
    #Grouping the data into chunks of size 2048.
    #So we're only training on a context size of 2048 for this quick run.
    tokens = sum(batch["input_ids"], [])
    #Flattens batch["input_ids"] (a list of lists) into one long list.
    n_full = len(tokens) // chunk * chunk
    return {"input_ids": [tokens[i : i+chunk] for i in range(0, n_full, chunk)]}
    #Batches back into multiples of chunk.

ds_lm = ds_tok.map(group_texts, batched=True)

In [ ]:
from datasets import IterableDataset

class TokenCappedDataset(IterableDataset):
    def __init__(self, base_ds, max_tokens: int):
        self.base_ds = base_ds
        self.max_tokens = max_tokens
        #We're certainly not going to go through the full datasets, so we'll cut off after a fixed number of tokens and call that an epoch.

    def __iter__(self):
        token_budget = self.max_tokens
        for ex in self.base_ds:
            tok_len = len(ex["input_ids"])
            if tok_len > token_budget:
                break
            token_budget -= tok_len
            yield ex

In [ ]:
from typing import List, Dict
import torch

class CausalLMCollator:
    def __init__(self, tokenizer, pad_to_multiple_of = 8):
        self.tok = tokenizer
        self.pad_multiple = pad_to_multiple_of

    def __call__(self, examples: List[Dict]) -> Dict[str, torch.Tensor]:
        ids = [e["input_ids"] for e in examples]

        batch = self.tok.pad({"input_ids": ids}, padding="longest", pad_to_multiple_of=self.pad_multiple, return_tensors="pt")
        return batch
        #Don't need "labels" in batch because we're not using Trainer; we're computing a custom loss.

In [ ]:
from torch.utils.data import DataLoader

collator = CausalLMCollator(tokenizer)

In [ ]:
import math
from tqdm.auto import tqdm
from pathlib import Path
from transformers.optimization import get_cosine_schedule_with_warmup

tokens_per_epoch = 100_000_000
num_epochs = 20
#I never hit the full 20 epochs since the loss function always seemed to converge comfortably before that.
#Also because training costs money.

handle = model.model.language_model.layers[23].register_forward_hook(hook_fn, with_kwargs=True)
##handle = model.model.language_model.layers[0].register_forward_hook(hook_fn, with_kwargs=True)
new_layer.train()

tokens_seen = 0
tokens_per_batch = 8 * 2048
steps_per_epoch = math.ceil(tokens_per_epoch / tokens_per_batch)

total_steps = num_epochs * steps_per_epoch
warmup_steps = int(0.05 * total_steps)

scheduler = get_cosine_schedule_with_warmup(optim, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

for epoch in range(num_epochs):
    epoch_ds = TokenCappedDataset(ds_lm, tokens_per_epoch)
    loader = DataLoader(epoch_ds, batch_size=8, collate_fn=collator)

    pbar = tqdm(total=steps_per_epoch, desc=f"Epoch {epoch:02d}", leave=False)
    epoch_loss_sum = 0.0

    for step, batch in enumerate(loader):
        batch = {k: v.to("cuda") for k, v in batch.items()}
        _ = model.model.language_model(**batch)
        #The hook takes care of everything.

        diff = pred_latent - gold_latent
        loss = diff.norm()
        #An L2-norm loss on the latents.

        optim.zero_grad()
        loss.backward()
        optim.step()
        scheduler.step()

        epoch_loss_sum += loss.item()
        pbar.update(1)
        pbar.set_postfix(loss=f"{loss.item():.4f}") 

        tokens_seen += tokens_per_batch

    pbar.close()
    avg_loss = epoch_loss_sum / steps_per_epoch
    print(f"Epoch {epoch}: Average Loss {avg_loss}")

    tokens_seen = 0

    ckpt_path = f"epoch_{epoch:02d}.pt"
    torch.save(
        {
            "epoch": epoch,
            "model_state_dict": new_layer.state_dict(),
            "optimizer_state_dict": optim.state_dict(),
            "avg_loss": avg_loss
        },
        ckpt_path
    )

handle.remove()